# Import Data File

In [1]:
import pandas as pd

df = pd.read_csv('Dataset_Scraping.csv')
df.head()

reviewId         userName  \
0  015c5548-10f6-4de1-a869-7ea9dbc884a5  Pengguna Google   
1  a351552b-9d3d-46a7-a9f4-500141b0214a  Pengguna Google   
2  88bce7dd-4db0-4007-b538-c434cc9aa288  Pengguna Google   
3  fb096ffa-dd1e-49d5-a376-d67d6c0cc6df  Pengguna Google   
4  8d2ef29d-42b9-4ce4-9d60-ce0399876e7e  Pengguna Google   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
1  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/EGemoI2N...   

                                             content  score  thumbsUpCount  \
0                                                 ok      5              0   
1                                        sangat baik      5              0   
2  Paket diproses terus gk pernah di kirim" baru ...      1              0   
3                          bagus tapi susah buat cod      2              0   
4  saldo gopay menguap secara ajaib setelah buka ...      1              0   

  reviewCreatedVersion                   at  \
0              3.305.0  2025-04-10 20:37:04   
1              3.305.0  2025-04-10 20:18:11   
2                  NaN  2025-04-10 20:09:16   
3                  NaN  2025-04-10 19:57:06   
4                3.163  2025-04-10 19:54:13   

                                        replyContent            repliedAt  \
0  Toppers, terima kasih atas rating yang kamu be...  2025-04-10 21:07:01   
1  Hi Toppers, terima kasih untuk rating dan ulas...  2025-04-10 20:47:01   
2  Hi Toppers, silakan sampaikan kritik/saran ata...  2025-04-10 20:47:03   
3  Halo Toppers, terima kasih atas rating yang ka...  2025-04-10 20:27:01   
4  Halo Toppers, terima kasih atas rating yang ka...  2025-04-10 20:27:02   

  appVersion  
0    3.305.0  
1    3.305.0  
2        NaN  
3        NaN  
4      3.163

# Pre-Processing Text

In [2]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
 
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text) 
    text = re.sub(r'RT[\s]', '', text)  
    text = re.sub(r"http\S+", '', text) 
    text = re.sub(r'[0-9]+', '', text)  
    text = re.sub(r'[^\w\s]', '', text) 
 
    text = text.replace('\n', ' ') 
    text = text.translate(str.maketrans('', '', string.punctuation))  
    text = text.strip(' ') 
    return text
 
def casefoldingText(text): 
    text = text.lower()
    return text
 
def tokenizingText(text): 
    text = word_tokenize(text)
    return text
 
def filteringText(text): 
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text
 
def stemmingText(text): 
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
 
    words = text.split()
 
    stemmed_words = [stemmer.stem(word) for word in words]
 
    stemmed_text = ' '.join(stemmed_words)
 
    return stemmed_text
 
def toSentence(list_words): 
    sentence = ' '.join(word for word in list_words)
    return sentence

In [3]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal" }
def fix_slangwords(text):
    words = text.split()
    fixed_words = []
 
    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)
 
    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [4]:
# Ganti NaN jadi string kosong
df['content'] = df['content'].fillna('')

# Pastikan semuanya string
df['content'] = df['content'].astype(str)

# Lanjutkan pembersihan teks
df['text_clean'] = df['content'].apply(cleaningText)

In [5]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
df['text_clean'] = df['content'].apply(cleaningText)
 
# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
df['text_casefoldingText'] = df['text_clean'].apply(casefoldingText)
 
# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
df['text_slangwords'] = df['text_casefoldingText'].apply(fix_slangwords)
 
# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
df['text_tokenizingText'] = df['text_slangwords'].apply(tokenizingText)
 
# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
df['text_stopword'] = df['text_tokenizingText'].apply(filteringText)
 
# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
df['text_akhir'] = df['text_stopword'].apply(toSentence)

# Pelabelan

In [6]:
import csv
import requests
from io import StringIO
 
# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()
 
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
 
if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma
 
    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")
 
# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()
 
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
 
if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma
 
    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [7]:
def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    
    # Tambah skor positif
    for word in text:
        if word in lexicon_positive:
            score += lexicon_positive[word]
    
# Kurangi skor negatif
    for word in text:
        if word in lexicon_negative:
            score -= abs(lexicon_negative[word])

    
    # Tentukan polaritas berdasarkan skor
    if score > 0:
        polarity = 'positive'
    elif score < 0:
        polarity = 'negative'
    else:
        polarity = 'neutral'
    
    return score, polarity

In [8]:
results = df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
df['polarity_score'] = results[0]
df['polarity'] = results[1]
print(df['polarity'].value_counts())

polarity
positive    110189
negative     69866
neutral      53945
Name: count, dtype: int64


# Ekstraksi Fitur

### Word Embedding 

In [9]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
text_data = df['text_akhir'].tolist()

In [11]:
tokenized_data = [word_tokenize(sentence.lower()) for sentence in text_data]

In [12]:
model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

In [13]:
word_vectors = model.wv

similar_words = word_vectors.most_similar('tokopedia', topn=5)
print("Kata-kata yang mirip dengan 'tokopedia':", similar_words)

vector = word_vectors['tokopedia']
print("Vektor untuk 'tokopedia':", vector)


Kata-kata yang mirip dengan 'tokopedia': [('tokped', 0.774524450302124), ('toped', 0.6884376406669617), ('ditokopedia', 0.6252616047859192), ('apapun', 0.5536631345748901), ('mengecewakan', 0.5423694252967834)]
Vektor untuk 'tokopedia': [ 1.2861452   0.33203995 -0.23369524  0.7796169  -0.56818074  1.1956966
  0.88587445 -0.34757835 -0.55521226 -0.9389649   0.2713204   0.67339844
 -0.23430406 -0.3793461  -0.34109652 -0.14620505  0.72882724 -1.3625083
 -0.16190735 -0.51088446  0.42297795  1.8247553   0.5324018   1.0264721
  1.2437156  -1.2455893   0.45674843  0.80333495 -1.5580342  -1.0227243
  0.9310755   0.37960264  1.101195   -0.90279436 -1.4851131  -0.0531421
  0.36331913 -0.91077316  1.0755975  -1.753191   -3.6680756  -0.5491072
 -0.5407712   1.0662955   0.63256764  0.5087877   0.05658465  1.2992185
 -0.07262778  0.61301064 -0.8549316   1.5688993  -0.7983181   0.47020265
  0.7143797  -0.83215785 -0.7488489   0.25286242  0.6178327   1.8127483
 -0.4420208   2.078489    0.08573525 -1.1

### Term Frequency-Inverse Document Frequency (TF-IDF)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
documents = df['text_akhir'].tolist()

In [16]:
tfidf_vectorizer = TfidfVectorizer()

In [17]:
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

In [18]:
print("Vocabulary:", tfidf_vectorizer.vocabulary_)

Vocabulary: {'ok': 51143, 'paket': 53042, 'diproses': 18317, 'gk': 24435, 'kirim': 34799, 'pakai': 52931, 'tokopedia': 73646, 'udh': 76327, 'kali': 31474, 'kayak': 32169, 'gini': 24293, 'rugi': 62018, 'bagus': 4642, 'susah': 69536, 'cod': 13705, 'saldo': 62633, 'gopay': 24885, 'menguap': 44901, 'ajaib': 962, 'buka': 11607, 'aplikasi': 3335, 'mencuri': 44152, 'pelanggan': 54240, 'melarikan': 43053, 'tanggung': 70349, 'menipu': 45021, 'mantap': 41631, 'separah': 65832, 'kurir': 36760, 'tgl': 72685, 'sampe': 63055, 'blm': 10369, 'hmmm': 27217, 'buta': 12048, 'map': 42160, 'seler²': 65125, 'terbaik': 71204, 'tksh': 73280, 'danax': 14877, 'mntap': 46613, 'android': 2400, 'instal': 28703, 'saran': 63374, 'tokped': 74290, 'mending': 44207, 'expedisi': 22127, 'jt': 30814, 'hapus': 26243, 'aja': 873, 'ngga': 49103, 'pesen': 57273, 'segitu': 64502, 'dikirim': 17354, 'gabisa': 23006, 'masuk': 42472, 'jos': 30698, 'sarankan': 63388, 'membeli': 43446, 'dimarketplace': 17699, 'inikurir': 28491, 'keo

In [19]:
print("Ukuran TF-IDF matrix:", tfidf_matrix.shape)
print("Contoh baris pertama (dense):", tfidf_matrix[0].toarray())

Ukuran TF-IDF matrix: (234000, 80676)
Contoh baris pertama (dense): [[0. 0. 0. ... 0. 0. 0.]]


# Data Split

In [20]:
from sklearn.model_selection import train_test_split

def split_data(X, y, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    return X_train, X_test, y_train, y_test


# Modeling

### Pelatihan Logistic Regression, Ekstraksi Fitur TF-IDF, Data Split 80:20

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Ekstraksi Fitur TF-IDF
tfidf = TfidfVectorizer()
texts = df['text_akhir']  
labels = df['polarity']   
X = tfidf.fit_transform(texts)

# Data Split 80:20
def split_data(X, y, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_data(X, labels)

# Pelatihan Logistic Regression
logistic_regression = LogisticRegression(max_iter=1000)  
logistic_regression.fit(X_train, y_train)

y_pred_train_lr = logistic_regression.predict(X_train)
y_pred_test_lr = logistic_regression.predict(X_test)

accuracy_train_lr = accuracy_score(y_pred_train_lr, y_train)
accuracy_test_lr = accuracy_score(y_pred_test_lr, y_test)

print('Logistic Regression - accuracy_train:', accuracy_train_lr)
print('Logistic Regression - accuracy_test:', accuracy_test_lr)


Logistic Regression - accuracy_train: 0.960491452991453
Logistic Regression - accuracy_test: 0.9402350427350428


### Pelatihan Logistic Regression, Ekstraksi Fitur CountVectorizer, Data Split 80:20

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

texts = df['text_akhir']
labels = df['polarity']

# Ekstraksi Fitur CountVectorizer
count_vect = CountVectorizer()
X = count_vect.fit_transform(texts)

# Split data 80:20
X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.2, random_state=42, stratify=labels
)

# Pelatihan Logistic Regression
logistic_regression = LogisticRegression(max_iter=1000)
logistic_regression.fit(X_train, y_train)

y_pred_train = logistic_regression.predict(X_train)
y_pred_test = logistic_regression.predict(X_test)

accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

print("Logistic Regression (CountVectorizer) - accuracy_train:", accuracy_train)
print("Logistic Regression (CountVectorizer) - accuracy_test:", accuracy_test)


Logistic Regression (CountVectorizer) - accuracy_train: 0.973525641025641
Logistic Regression (CountVectorizer) - accuracy_test: 0.9490598290598291


### Pelatihan Decision Tree, Ekstraksi Fitur TF-IDF, Data Split 70:30

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Ekstraksi Fitur TF-IDF
tfidf = TfidfVectorizer()
texts = df['text_akhir']
labels = df['polarity']
X = tfidf.fit_transform(texts)

# Data Split 70:30
def split_data(X, y, test_size=0.3, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_data(X, labels)

# Pelatihan Decision Tree
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)

y_pred_train_dt = decision_tree.predict(X_train)
y_pred_test_dt = decision_tree.predict(X_test)

accuracy_train_dt = accuracy_score(y_pred_train_dt, y_train)
accuracy_test_dt = accuracy_score(y_pred_test_dt, y_test)

print('Decision Tree - accuracy_train:', accuracy_train_dt)
print('Decision Tree - accuracy_test:', accuracy_test_dt)


Decision Tree - accuracy_train: 1.0
Decision Tree - accuracy_test: 0.8731623931623932


### Pelatihan Decision Tree, Ekstraksi Fitur CountVectorizer, Data Split 70:30

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Ekstraksi Fitur CountVectorizer
count_vect = CountVectorizer()
texts = df['text_akhir']
labels = df['polarity']
X = count_vect.fit_transform(texts)

# Data Split 70:30
def split_data(X, y, test_size=0.3, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_data(X, labels)

# Pelatihan Decision Tree
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)

y_pred_train_dt = decision_tree.predict(X_train)
y_pred_test_dt = decision_tree.predict(X_test)

accuracy_train_dt = accuracy_score(y_pred_train_dt, y_train)
accuracy_test_dt = accuracy_score(y_pred_test_dt, y_test)

print('Decision Tree (CountVectorizer) - accuracy_train:', accuracy_train_dt)
print('Decision Tree (CountVectorizer) - accuracy_test:', accuracy_test_dt)


Decision Tree (CountVectorizer) - accuracy_train: 1.0
Decision Tree (CountVectorizer) - accuracy_test: 0.8812535612535612


# Inference

### Uji Model

In [25]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['text_akhir'])


In [26]:
# Inference
teks_baru = [
    "Barang sesuai dengan deskripsi, kualitas oke",
    "Pelayanan buruk",
    "Pengiriman cepat tapi kualitas produk biasa saja."
]

X_baru = tfidf.transform(teks_baru)
prediksi_kelas = logistic_regression.predict(X_baru)

for teks, prediksi in zip(teks_baru, prediksi_kelas):
    print(f"Ulasan: '{teks}'\n=> Kelas Prediksi: {prediksi}\n")


Ulasan: 'Barang sesuai dengan deskripsi, kualitas oke'
=> Kelas Prediksi: positive

Ulasan: 'Pelayanan buruk'
=> Kelas Prediksi: negative

Ulasan: 'Pengiriman cepat tapi kualitas produk biasa saja.'
=> Kelas Prediksi: positive

